We have obtained two sets of data: One for finding the set of skills that are required for each job and the other for obtaining the skills taught in each course

In [ ]:
##Importing the data file from google drive

from google.colab import drive
drive.mount('/content/drive/')          # this will direct you to a link where you can get anmhkk authorization key
import sys
sys.path.append('/content/drive/My Drive/DS')

Mounted at /content/drive/


In [ ]:
##Changing the working directory

%cd '/content/drive/My Drive/DS'

/content/drive/My Drive/DS


In [ ]:
ls

CoursetoSkills.csv  indeed_job_dataset.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk 
from bs4 import BeautifulSoup
import copy

Let's have a look on the dataset obtained from job postings on indeed

In [ ]:
df= pd.read_csv('indeed_job_dataset.csv')
df.head()

,Unnamed: 0,Job_Title,Link,Queried_Salary,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,Date_Since_Posted,Description,Location,Company_Revenue,Company_Employees,Company_Industry,python,sql,machine learning,r,hadoop,tableau,sas,spark,java,Others,CA,NY,VA,TX,MA,IL,WA,MD,DC,NC,Other_states,Consulting and Business Services,Internet and Software,Banks and Financial Services,Health Care,Insurance,Other_industries
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']",2,Express Scripts,3301.0,3.3,1.0,"[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ...",MO,More than $10B (USD),"10,000+",Health Care,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5,Money Mart Financial Services,NaN,NaN,15.0,"[<p><b>What do we need?</b></p>, <ul><li>\r\r\...",TX,NaN,NaN,NaN,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...",9,comScore,62.0,3.5,1.0,"[<ul><li>Validate, analyze, and conduct statis...",OR,NaN,NaN,NaN,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],1,Central Intelligence Agency,158.0,4.3,30.0,"[<p>Full time</p>, <p>Washington, DC metro are...",DC,NaN,NaN,Government,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",7,Federal Reserve Bank of Dallas,495.0,4.1,30.0,[<ul><li>Assist in consultations with business...,TX,NaN,"Less than 10,000",Banks and Financial Services,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


We see that there are various information obtained from job posings of different job titles. The columns of the dataset that are of our interest are the 'Job_Title' and 'Skill'. We drop nan values from these two columns and reset the indices for further processing

In [ ]:
df.dropna(subset=['Job_Title', 'Skill'],inplace=True)
df.reset_index(inplace=True, drop=True)

Here, we form a dictionary of the jobs and their required skills. Keys of the dictionary are the names of the courses and values are the list of skills that are required from the dataset

In [ ]:
import ast

dict1={}
rows=df.shape[0]
new_df=df

for row in range(rows):
  job=new_df['Job_Title'][row]
  if job not in dict1.keys():
    list_of_skills=ast.literal_eval(df['Skill'][row])
    dict1[job]=list_of_skills
  else:
    list_of_skills=ast.literal_eval(df['Skill'][row])
    for Skill in list_of_skills:
      if Skill not in dict1[job]:
        dict1[job].append(Skill)

##dict is a dictionary with job title as key and skills as the values
## it has information about job titles acquired from job postings in indeed

We have formed another dataset by manually inspecting course descriptions and syllabuses of the list of elective courses of the program. Using the similar manner, we form a dictionary with the course titles as the keys and the list of skills taught in each course as the values

In [ ]:
df2= pd.read_csv('CoursetoSkills.csv')
dict2={}
rows2=df2.shape[0]



for row in range(rows2):
  course=df2['Course'][row]
  if course not in dict2.keys():
    skills=ast.literal_eval(df2['Skill'][row])
    dict2[course]=skills
  else:
    skills=ast.literal_eval(df2['Skill'][row])
    for skill in skills:
      if skill not in dict2[course]:
        dict2[course].append(skill)

##dict2 is a dictionary with course name as key and skills as the values

Here, we form a set of all skills that are either taught in the courses or required in the job postings. 

In [ ]:
all_skill = []
for course in dict2:
  for skill in dict2[course]:
    if skill not in all_skill:
      all_skill.append(skill)


There is a total number of 161 skills that we are dealing with

In [ ]:
len(all_skill)

161

In [ ]:
for job in dict1:
  dict1[job]=list(set(dict1[job]).intersection(set(all_skill)))


Let's have a look on the job titles that we have

In [ ]:
for key in dict1:
  print(key)

Data Scientist
Graduate Studies Program - Data Scientist
Data Scientist I
Data Scientist - Entry to Experienced Level
Geospatial Data Scientist
Bioinformatics Data Scientist
Jr. Data Scientist - Machine Learning
Enrollment Data Scientist & Econometrician
Junior Data Scientist
Data Analytics and Machine Learning Specialist
Postdoctoral Associate/Data Scientist, Laboratory for Multis...
innovationOchsner (iO) - Data Scientist - Benson Tower
Statistician/Data Scientist
Statistician / Data Scientist
Environmental Data Scientist, Bureau of Environmental Survei...
Environmental Data Manager/Data Scientist (Associate Level)
Sr. Data Scientist - Secret
innovation Ochsner - iO - Data Scientist - Benson Tower
Data Analysis & Machine Learning Postdoctoral Researcher
Applied Machine Learning Scientist - Digital Imaging
Machine Learning Scientist
Data Scientist/Sr. Scientific Programmer (Researcher 6)
Machine Learning Intern (Summer 2019)
Data Scientist (Research Director)
Machine Learning Scientis

#Recommendation part


In [ ]:

courses=copy.deepcopy(dict2)
jobs_all=copy.deepcopy(dict1)

As an example of the skills in each dataset, let's print the set of skills in one sample course

In [ ]:
courses['MIE1628']

['Big Data',
 'data analytics',
 'Python',
 'Hadoop',
 'MapReduce',
 'Spark',
 'Cloud',
 'Microsoft Azure',
 'Amazon Web Services',
 'Google Cloud',
 'Real-time Analytics',
 ' Machine Learning',
 'SQL']

This function compares the set of skills that the user already has and the skills required by the job. We will look for courses with the required skills later

In [ ]:
def skills_needed_calculator(skills_possesed,jobs_skills_input):
  skills_needed=list(set(jobs_skills_input) - set(skills_possesed))
  return skills_needed

In [ ]:
skills_needed_calculator([1,2,5,7],[1,2,3,4,5])

[3, 4]

This function calculates a score for a given course based on the number of required skills that the course covers

In [ ]:
def course_score(skills_needed,course_skills):
  skills_needed = set(skills_needed)
  course_skills = set(course_skills)
  learned_skills = skills_needed.intersection(course_skills)
  score = len(learned_skills)
  return score

Utilizing the functions introduced, we form a course_finder that takes the job titles, looks for the required skills and keeps it. Also, it takes the list of skills that the user has already obtained. It looks for the courses that cover most of the deficiencies of the user's skill set. Since we don't want to offer several courses with the same and repeated set of skills, we remove the skills taught in each recommended course from the set of required skills.

In [ ]:
def course_finder(job_title,skill_list,jobs_all,courses):
  jobs = copy.deepcopy(jobs_all)
  skills_possesed = copy.deepcopy(skill_list)
  skills_needed = skills_needed_calculator(skills_possesed,jobs[job_title])
  courses_to_take = []
  course_scores = [[x,0] for x in courses]
  while len(skills_needed) != 0:
    for course in course_scores:
      course[1] = course_score(skills_needed,courses[course[0]])
    course_scores.sort(key=lambda x: x[1],reverse=True)
    courses_to_take.append(course_scores[0][0])
    for item in courses[course_scores[0][0]]:
      if item not in skills_possesed:
        skills_possesed.append(item)
    skills_needed = skills_needed_calculator(skills_possesed,skills_needed)
    course_scores.remove(course_scores[0])
  return courses_to_take

Definitely, not all requirements of all jobs are covered by UofT courses and so, the user must be notified and warned about the this. This function, tells the user what skills are still remaining after taking those recommended courses so that the user can either self-study it or think of another solution.

In [ ]:
def do_courses_offer_all_skills_needed(skills_needed_input,courses_input):
  skills_needed_but_not_offered = copy.deepcopy(skills_needed_input)
  courses = copy.deepcopy(courses_input)
  # print(skills_needed_but_not_offered)
  for course in courses:
    # print(course,' ',skills_needed_but_not_offered)
    for skill in skills_needed_but_not_offered :
      for skill_in_course in courses[course]:
        if skill in skill_in_course:
          skills_needed_but_not_offered.remove(skill)
  return skills_needed_but_not_offered
  

In [ ]:
max_len = 0
for job in jobs_all:
  if len(jobs_all[job]) > max_len:
    max_len = len(jobs_all[job])
max_len

15

Let's assume we have a user who has not had any previous experience with the requirements of a 'Data Scinetist' job title. Here is the list of courses that we recommend to him.

In [ ]:
course_finder('Data Scientist',[],dict1,dict2)

['MIE1628',
 'MIE1622',
 'APS1002',
 'MIE1413',
 'APS1080',
 'APS1040',
 'ECE 1778',
 'ECE 1779',
 'MIE1512']

As another example, a user with no previous skills wants to become a 'Sr Big Data Engineer'. These are the courses that s/he should take

In [ ]:
course_finder('Sr Big Data Engineer',[],dict1,dict2)

['MIE1628', 'MIE1622']